In [75]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqlalchemy import create_engine
import matplotlib as mpl
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
%matplotlib inline

In [76]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path)
cur = con.cursor()
defender = pd.read_sql_query("SELECT *  FROM avgPlayerStatistics AS ps WHERE ps.player_position = 'D' ;", con,)

In [77]:
pd.set_option('display.max_columns', None)
defender.head()

,index,assists,blocks,commited_fouls,commited_penalty,conceded,drawn_fouls,dribble_attempts,dribble_past,dribble_success,fixture_id,interceptions,key_passes,minutes,missed_penalty,number,offside,on_goal_shots,pass_accuracy,player_id,player_position,rating,red,saved_penalty,saves,scored_penalty,timestamp,total_duels,total_goals,total_passes,total_shots,total_tackels,won_duels,won_penalty,yellow
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,185545,2.0,0.0,90.0,0.0,5.0,0.0,0.0,83.0,6366,D,6.7,0.0,0.0,0.0,0.0,1438972200,3.0,0.0,31.0,0.0,1.0,2.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,185545,3.0,0.0,90.0,0.0,19.0,0.0,0.0,82.0,102,D,6.2,0.0,0.0,0.0,0.0,1438972200,18.0,0.0,37.0,0.0,2.0,6.0,0.0,1.0
2,4,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,185545,3.0,0.0,90.0,0.0,28.0,0.0,0.0,83.0,2725,D,7.0,0.0,0.0,0.0,0.0,1438972200,10.0,0.0,36.0,1.0,2.0,4.0,0.0,1.0
3,8,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,185545,1.0,0.0,45.0,0.0,6.0,0.0,0.0,94.0,2049,D,7.0,0.0,0.0,0.0,0.0,1438972200,2.0,0.0,17.0,0.0,1.0,2.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,185545,3.0,0.0,90.0,0.0,17.0,0.0,0.0,93.0,87211,D,6.8,0.0,0.0,0.0,0.0,1438972200,6.0,0.0,31.0,0.0,2.0,2.0,0.0,0.0


In [78]:
df = defender[['fixture_id','player_id','won_duels','blocks', 'total_tackels', 'pass_accuracy', 'key_passes', 'total_passes', 'dribble_success', 'interceptions' ,'total_duels', 'drawn_fouls', 'minutes']]
#display(df)
df.head()


,fixture_id,player_id,won_duels,blocks,total_tackels,pass_accuracy,key_passes,total_passes,dribble_success,interceptions,total_duels,drawn_fouls,minutes
0,185545,6366,2.0,0.0,1.0,83.0,0.0,31.0,0.0,2.0,3.0,0.0,90.0
1,185545,102,6.0,0.0,2.0,82.0,0.0,37.0,2.0,3.0,18.0,0.0,90.0
2,185545,2725,4.0,2.0,2.0,83.0,0.0,36.0,1.0,3.0,10.0,1.0,90.0
3,185545,2049,2.0,1.0,1.0,94.0,0.0,17.0,0.0,1.0,2.0,1.0,45.0
4,185545,87211,2.0,0.0,2.0,93.0,0.0,31.0,0.0,3.0,6.0,0.0,90.0


In [79]:
for key in df.keys(): # check for equality of dataframes
    print(key,":", defender[key].equals(df[key]))

fixture_id : True
player_id : True
won_duels : True
blocks : True
total_tackels : True
pass_accuracy : True
key_passes : True
total_passes : True
dribble_success : True
interceptions : True
total_duels : True
drawn_fouls : True
minutes : True


In [80]:
# NORMALIZE ALLE AUßER FIXTURE_ID

x = df.to_numpy() #returns a numpy array
x1 = x[:, 2:] # every column from colum (player_id)


robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x1)
x_scaled = np.concatenate(( x[:,[0,1]], x_scaled), axis=1)
final = pd.DataFrame(x_scaled, columns=df.columns)
columns = df.keys()
for i in range(2): # fixture_id bis timestamp zu int
    final[columns[i]] = pd.to_numeric(final[columns[i]], downcast='integer')
display(final)

,fixture_id,player_id,won_duels,blocks,total_tackels,pass_accuracy,key_passes,total_passes,dribble_success,interceptions,total_duels,drawn_fouls,minutes
0,185545,6366,-0.714286,0.000000,0.285714,0.395833,-0.333333,-0.052632,-0.333333,0.500000,-1.000000,-0.9,0.042254
1,185545,102,0.714286,0.000000,1.000000,0.333333,-0.333333,0.263158,3.000000,1.333333,2.571429,-0.9,0.042254
2,185545,2725,0.000000,5.000000,1.000000,0.395833,-0.333333,0.210526,1.333333,1.333333,0.666667,0.6,0.042254
3,185545,2049,-0.714286,2.500000,0.285714,1.083333,-0.333333,-0.789474,-0.333333,-0.333333,-1.238095,0.6,-3.126761
4,185545,87211,-0.714286,0.000000,1.000000,1.020833,-0.333333,-0.052632,-0.333333,1.333333,-0.285714,-0.9,0.042254
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113646,705683,9828,-1.285714,0.000000,-0.428571,-4.654167,0.000000,-1.515789,0.000000,-0.833333,-1.428571,-0.9,-5.746479
113647,705683,41906,1.071429,0.000000,0.285714,-3.416667,6.333333,-0.157895,1.333333,-0.333333,0.666667,2.1,0.042254
113648,705683,41609,-0.357143,0.833333,0.047619,-0.687500,-0.333333,-0.842105,-0.333333,0.500000,-0.206349,-0.4,-3.478873
113649,705683,95252,-0.428571,0.000000,-0.142857,-0.916667,-0.333333,-0.631579,-0.333333,0.833333,-0.761905,-0.6,-0.591549


In [81]:
#NORMALIZE ALL FOR COMPARISON

x = df.to_numpy() #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x)
all_norm = pd.DataFrame(x_scaled, columns=df.columns)

In [82]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
final[x] = pd.to_numeric(final[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(final[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_18648/3261938473.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [83]:
for key in df.keys():
    print(key,":", all_norm[key].equals(final[key]))

fixture_id : False
player_id : False
won_duels : True
blocks : True
total_tackels : True
pass_accuracy : True
key_passes : True
total_passes : True
dribble_success : True
interceptions : True
total_duels : True
drawn_fouls : True
minutes : True


In [84]:
weights = [1,1,0.15,0.15,0.15,0.1,0.1,0.1,0.05,0.05,0.05,0.05,0.05]
print(len(weights))
final_mul_weights = final.mul(weights)
final_mul_weights['fixture_id'] = pd.to_numeric(final_mul_weights['fixture_id'], downcast='integer')
final_mul_weights['player_id'] = pd.to_numeric(final_mul_weights['player_id'], downcast='integer')  #um equals true zu erreichen müssen fixture id gleichen typ haben
print("fixture_id:", final['fixture_id'].equals(final_mul_weights['fixture_id']))
print("player_id:", final['player_id'].equals(final_mul_weights['player_id']))
final_mul_weights.head()

13
fixture_id: True
player_id: True


,fixture_id,player_id,won_duels,blocks,total_tackels,pass_accuracy,key_passes,total_passes,dribble_success,interceptions,total_duels,drawn_fouls,minutes
0,185545,6366,-0.107143,0.000,0.042857,0.039583,-0.033333,-0.005263,-0.016667,0.025000,-0.050000,-0.045,0.002113
1,185545,102,0.107143,0.000,0.150000,0.033333,-0.033333,0.026316,0.150000,0.066667,0.128571,-0.045,0.002113
2,185545,2725,0.000000,0.750,0.150000,0.039583,-0.033333,0.021053,0.066667,0.066667,0.033333,0.030,0.002113
3,185545,2049,-0.107143,0.375,0.042857,0.108333,-0.033333,-0.078947,-0.016667,-0.016667,-0.061905,0.030,-0.156338
4,185545,87211,-0.107143,0.000,0.150000,0.102083,-0.033333,-0.005263,-0.016667,0.066667,-0.014286,-0.045,0.002113


In [85]:
fixtures = final_mul_weights.pop('fixture_id')
players = final_mul_weights.pop('player_id')


In [86]:
fixtures.head()

0    185545
1    185545
2    185545
3    185545
4    185545
Name: fixture_id, dtype: int32

In [87]:
test = final_mul_weights.sum(axis=1)
display(test)
test.head()
player_rating = pd.Series(test,
              name="d_rating")
player_rating = player_rating.to_frame()
print(type(player_rating))

0        -0.147853
1         0.585809
2         1.126082
3         0.085191
4         0.099171
            ...   
113646   -1.319558
113647    0.669895
113648   -0.303650
113649   -0.346545
113650    0.380484
Length: 113651, dtype: float64

<class 'pandas.core.frame.DataFrame'>


In [88]:
player_rating.insert(0, 'fixture_id', fixtures)
player_rating.insert(1, 'player_id', players)

In [89]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
player_rating[x] = pd.to_numeric(player_rating[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(player_rating[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_18648/2261603312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [90]:
player_rating.describe()
pd.get_option("display.max_rows")
pd.set_option("display.max_rows", 20)

In [104]:
engine = create_engine(r'sqlite:///C:\Users\Anwender\Dropbox\PC\Documents\GitHub\sql_dataset_creation\football.db', echo=False)
player_rating.to_sql('defenderRating', con=engine, if_exists='append') #if_exists='append'

In [92]:
test = pd.read_sql_query("SELECT DISTINCT player_id FROM defenderRating WHERE player_id IN (SELECT player_id FROM defenderRating ORDER BY d_rating DESC LIMIT 20);", con)
display(test)

,player_id
0,97864
1,21490
2,20974
3,860
4,18771
5,18963
6,37450
7,129782


In [102]:
test = pd.read_sql_query("SELECT  * FROM lineups WHERE player_id = 129782 AND player_position = 'D';", con)
display(test)

,fixture_id,team_id,timestamp,player_id,player_name,player_number,player_position,grid
0,601089,214,1606762800,129782,Léo Andrade,66,D,3:3
1,601106,214,1608390000,129782,Léo Andrade,66,D,3:4
2,601119,214,1609181100,129782,Léo Andrade,66,D,3:3
3,601124,214,1609686000,129782,Léo Andrade,66,D,3:3
4,601134,214,1610053200,129782,Léo Andrade,66,D,3:3
...,...,...,...,...,...,...,...,...
16,601277,214,1619359200,129782,Léo Andrade,66,D,3:4
17,601287,214,1619724600,129782,Léo Andrade,66,D,3:3
18,601291,214,1620237600,129782,Léo Andrade,66,D,3:3
19,601305,214,1620568800,129782,Léo Andrade,66,D,2:1


In [94]:
test = pd.read_sql_query("SELECT player_id FROM defenderRating", con)
display(test)

,player_id
0,6366
1,102
2,2725
3,2049
4,87211
...,...
227297,9828
227298,41906
227299,41609
227300,95252


In [110]:
test = pd.read_sql_query("SELECT DISTINCT * FROM goalkeeperRating AS gr INNER JOIN defenderRating AS dr ON gr.fixture_id = dr.fixture_id WHERE gr.fixture_id = 185545", con)
display(test)
# defenderRating
# midfieldRating
# forwardRating

,index,fixture_id,player_id,g_rating,index,fixture_id,player_id,d_rating
0,0,185545,87166,-0.483114,0,185545,6366,-0.147853
1,0,185545,87166,-0.483114,1,185545,102,0.585809
2,0,185545,87166,-0.483114,2,185545,2725,1.126082
3,0,185545,87166,-0.483114,3,185545,2049,0.085191
4,0,185545,87166,-0.483114,4,185545,87211,0.099171
5,0,185545,87166,-0.483114,5,185545,259,0.815885
6,0,185545,87166,-0.483114,6,185545,2283,0.305264
7,0,185545,87166,-0.483114,7,185545,163,0.936279
8,1,185545,1800,-0.731252,0,185545,6366,-0.147853
9,1,185545,1800,-0.731252,1,185545,102,0.585809
